# Cleaning and Formatting the Datasets
Cleaning up the Data from each quarterly report with respect to its stock ticker.

### Importing Libraries and Data

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 70)
import numpy as np
from tqdm import tqdm_notebook as tqdm
import _pickle as pickle

In [2]:
with open("stockpup.pkl",'rb') as fp:
    stocks_df = pickle.load(fp)

## Preprocessing the Data

### Setting the Index to the Date

In [31]:
def setting_index(df):
    """
    Returns a sorted datetime index
    """
    df['Quarter end'] = pd.to_datetime(df['Quarter end'])
    df.set_index("Quarter end", inplace=True)
    return df.sort_index(ascending=True)

for i in tqdm(stocks_df.keys()):
    stocks_df[i] = setting_index(stocks_df[i])

### Replacing all "None" values with zero

In [32]:
for i in tqdm(stocks_df.keys()):
    stocks_df[i].replace("None", 0, inplace=True)

### Converting all values to numeric values

In [33]:
# Creating a new dictionary that contains the numerical values
num_df = {}

for i in tqdm(stocks_df.keys()):
    num_df[i] = stocks_df[i].apply(pd.to_numeric)

### Replacing values with percent difference 
(Between each quarter)

Also, mutliplying by 100 for better readability

In [34]:
pcnt_df = {}

for i in tqdm(num_df.keys()):
    pcnt_df[i] = num_df[i].pct_change(periods=1).apply(lambda x: x*100)

#### Replacing infinite values with NaN

In [35]:
for i in tqdm(pcnt_df.keys()):
    pcnt_df[i] = pcnt_df[i].replace([np.inf, -np.inf], np.nan)

## Creating the Classes
- Buy (because the highest high and lowest low of the quarter will both increase by 3% or more)
- Sell (because the lowest low and highest high of the quarter will both decrease by 3% or more)
- Hold (because it will not do either)

In [38]:
def class_creation(df, thres=3):
    """
    Creates classes of:
    - buy(1)
    - hold(2)
    - sell(0)
    
    Threshold can be changed to fit whatever price change is desired
    """
    if df['Price high'] >= thres and df['Price low'] >= thres:
        # Buys
        return 1
    
    elif df['Price high'] <= -thres and df['Price low'] <= -thres:
        # Sells
        return 0
    
    else:
        # Holds
        return 2

Creating a new DataFrame that contains the class 'Decision' determining if a quarterly reports improvement is a buy, hold, or sell.

In [39]:
new_df = {}

for i in tqdm(pcnt_df.keys()):
    # Assigning the the new DF
    new_df[i] = pcnt_df[i]
    
    # Creating the new column with the classes, shifted by -1 in order to know if the prices will increase/decrease in the next quarter.
    new_df[i]['Decision'] = new_df[i].apply(class_creation, axis=1).shift(-1)

### Excluding the first and last rows
This is done because the last row has no data to compare percent improvements to and the first row does not have any data to show if the price will increase in the future.

In [40]:
for i in tqdm(new_df.keys()):
    new_df[i] = new_df[i][1:-1]

#### Examining an example DF to check if the classes were assigned correctly

In [41]:
new_df['A'][['Price high', 'Price low', 'Decision']]

,Price high,Price low,Decision
Quarter end,,,
2000-04-30,100.630915,77.500000,0.0
2000-07-31,-36.635220,-42.605634,0.0
2000-10-31,-37.468983,-4.760736,2.0
2001-01-31,7.936508,-1.932492,0.0
2001-04-30,-19.117647,-34.314241,2.0
2001-07-31,-25.127273,4.800000,0.0
2001-10-31,-20.592521,-31.297710,2.0
2002-01-31,1.834862,22.555556,1.0
2002-04-30,14.114114,12.556664,0.0


### Combining all stock DFs into one

In [42]:
big_df = pd.DataFrame()

for i in tqdm(pcnt_df.keys()):
    big_df = big_df.append(new_df[i], sort=False)

#### Quick check for NaN values

In [43]:
big_df.isna().sum()

Shares                                            0
Shares split adjusted                             0
Split factor                                      0
Assets                                           27
Current Assets                                10007
Liabilities                                      27
Current Liabilities                           10028
Shareholders equity                              27
Non-controlling interest                      37981
Preferred equity                              54063
Goodwill & intangibles                        14607
Long-term debt                                10956
Revenue                                         799
Earnings                                        638
Earnings available for common stockholders      624
EPS basic                                       885
EPS diluted                                    5626
Dividend per share                            20688
Cash from operating activities                  945
Cash from in

#### Filling the NaNs with 0

In [44]:
big_df.fillna(0, inplace=True)

In [45]:
# Checking the DF again for NaN
big_df.isna().sum()

Shares                                        0
Shares split adjusted                         0
Split factor                                  0
Assets                                        0
Current Assets                                0
Liabilities                                   0
Current Liabilities                           0
Shareholders equity                           0
Non-controlling interest                      0
Preferred equity                              0
Goodwill & intangibles                        0
Long-term debt                                0
Revenue                                       0
Earnings                                      0
Earnings available for common stockholders    0
EPS basic                                     0
EPS diluted                                   0
Dividend per share                            0
Cash from operating activities                0
Cash from investing activities                0
Cash from financing activities          

### Resetting the index
Because we no longer need the dates

In [46]:
big_df.reset_index(drop=True, inplace=True)
big_df.head()

,Shares,Shares split adjusted,Split factor,Assets,Current Assets,Liabilities,Current Liabilities,Shareholders equity,Non-controlling interest,Preferred equity,Goodwill & intangibles,Long-term debt,Revenue,Earnings,Earnings available for common stockholders,EPS basic,EPS diluted,Dividend per share,Cash from operating activities,Cash from investing activities,Cash from financing activities,Cash change during period,Cash at end of period,Capital expenditures,Price,Price high,Price low,ROE,ROA,Book value of equity per share,P/B ratio,P/E ratio,Cumulative dividends per share,Dividend payout ratio,Long-term debt to equity ratio,Equity to assets ratio,Net margin,Asset turnover,Free cash flow per share,Current ratio,Decision
0,0.060170,0.060170,0.0,3.011116,1.505420,2.212896,5.757136,3.477486,0.0,0.0,0.000000,0.0,10.641140,26.717557,26.717557,23.333333,20.000000,0.0,-109.895833,63.942308,-100.922819,-128.508772,-28.508772,216.666667,92.855945,100.630915,77.500000,0.000000,0.0,3.427419,0.000000,0.000000,0.0,0.0,0.0,0.459442,0.000000,0.00000,-137.179487,-4.020246,0.0
1,0.164196,0.164196,0.0,6.911624,5.675302,9.182531,14.821592,5.601034,0.0,0.0,0.000000,0.0,7.444668,-6.626506,-6.626506,-8.108108,-5.555556,0.0,-281.578947,22.287390,-763.636364,-29.487179,-28.118609,133.684211,-38.478261,-36.635220,-42.605634,0.000000,0.0,5.458090,-40.465919,0.000000,0.0,0.0,0.0,-1.230090,0.000000,0.00000,17.241379,-7.966629,0.0
2,0.739888,0.739888,0.0,7.640220,5.819611,8.034188,9.880478,7.405141,0.0,0.0,0.000000,0.0,26.292135,96.774194,96.774194,97.058824,91.176471,0.0,513.043478,-63.788969,-71.232877,-206.545455,41.678521,59.909910,-28.042403,-37.468983,-4.760736,0.000000,0.0,6.654344,-31.739130,0.000000,0.0,0.0,0.0,-0.223535,0.000000,0.00000,-144.117647,-3.696398,2.0
3,0.088384,0.088384,0.0,9.293769,-3.430592,16.044304,11.312545,5.242165,0.0,0.0,117.773788,0.0,-15.747331,-49.508197,-49.508197,-49.253731,-49.230769,0.0,-146.572104,476.158940,2300.000000,-292.150171,-56.526104,-67.887324,4.164211,7.936508,-1.932492,0.000000,0.0,5.112652,-2.335456,0.000000,0.0,0.0,0.0,-3.696591,0.000000,0.00000,-553.333333,-13.246196,0.0
4,0.066338,0.066338,0.0,-1.390096,-8.478301,-5.535860,0.912052,1.353546,0.0,0.0,2.061006,0.0,-16.156283,-37.662338,-37.662338,-38.235294,-36.363636,0.0,-100.000000,-100.000000,-100.000000,-166.785080,86.836028,199.122807,-24.570998,-19.117647,-34.314241,-13.111546,0.0,1.319044,-28.260870,-25.917859,0.0,0.0,0.0,2.775008,-8.163265,-6.47482,-100.000000,-9.304025,2.0


### Dropping the Prices columns
- Price
- Price high
- Price low

To prevent any data leakage because we are looking mainly at the QR's value changes rather than prices.

In [47]:
big_df.drop(['Price', 'Price high', 'Price low'], 1, inplace=True)

### Counting how many classes there are

In [48]:
big_df['Decision'].value_counts()

2.0    26426
1.0    22127
0.0    11616
Name: Decision, dtype: int64

Unequal classes are fine because we will use a specific evaluation metric to determine success in classification.

### Exporting the final dataframe

In [49]:
with open("main_df.pkl", 'wb') as fp:
    pickle.dump(big_df, fp)